In [ ]:
import tabula
import os

pdf_path = 'WI.pdf'
output_csv_path = 'page_2_tables.csv'
page_number = 2

tabula.convert_into(
    pdf_path,
    output_csv_path,
    output_format='csv',
    pages=str(page_number),
    lattice=True
)



Successfully extracted tables from page 2 and saved to page_2_tables.csv


In [21]:
import pdfplumber
import pandas as pd

pdf_path = 'WI.pdf'
page_number = 2

# Open the PDF and analyze page 2
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number-1]
    
    # Extract text to understand the structure
    text = page.extract_text()
    print("Text content:")
    print(text)
    
    # Extract words with their positions
    words = page.extract_words()
    print("\nWord positions:")
    for word in words[:10]:  # Print first 10 words as example
        print(f"Text: {word['text']}, x0: {word['x0']}, top: {word['top']}")
    
    # Try to detect table boundaries
    tables = page.find_tables()
    print("\nDetected table boundaries:")
    for table in tables:
        print(f"Table bounds: {table.bbox}")

Text content:
Wisconsin Medicaid, BadgerCare Plus Standard, and SeniorCare Preferred Drug List – Quick Reference
Revised 05/29/2025 Effective 05/01/2025
Acne Agents, Topical Analgesics/Anesthetics, Topical (cont) Analgesics, Opioids Long-Acting (cont) Androgenic Agents (cont)
adapalene 0.1% cream P lidocaine 5% ointment P oxycodone ER NP Depo-testosterone* P
adapalene OTC 0.1% gel P lidocaine 5% trans patch P oxymorphone ER NP methyltestosterone capsule NP
adapalene 0.3% gel P diclofenac 1.3% patch tramadol ER cap (Gen-Conzip) SCN NP testosterone gel packet
NP NP
benzoyl peroxide OTC 2.5%, (Gen-Flector) tramadol ER tab (Gen-Ryzolt) NP (Gen-Androgel)
SCN P
5%, 10% diclofenac 1.5% solution Belbuca Film NP testosterone pump
NP NP
clindamycin/benzoyl peroxide (Gen-Pennsaid solution) Conzip SCN NP (Gen-Axiron and Fortesta)
P
(Gen-Duac) diclofenac 2% pump Oxycontin NP Azmiro* NP
NP
clindamycin gel (Gen-Cleocin T) P (Gen-Pennsaid pump) Analgesics, Opioids Short-Acting Jatenzo SCN NP
clindamyc

In [22]:
import pdfplumber
import pandas as pd
import re

def extract_tables_from_pdf(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Extract all words with their positions
        words = page.extract_words(
            x_tolerance=3,
            y_tolerance=3,
            keep_blank_chars=True,
            use_text_flow=True,
            horizontal_ltr=True
        )
        
        # Group words by their vertical position (y-coordinate)
        # This helps identify rows
        rows = {}
        for word in words:
            # Round the y-coordinate to handle slight misalignments
            y_pos = round(word['top'])
            if y_pos not in rows:
                rows[y_pos] = []
            rows[y_pos].append(word)
        
        # Sort rows by y-position
        sorted_rows = sorted(rows.items())
        
        # Process each row
        data = []
        current_category = None
        
        for y_pos, row_words in sorted_rows:
            # Skip header rows
            if y_pos < 100:  # Adjust this threshold based on your PDF
                continue
                
            # Sort words in the row by x-position
            row_words.sort(key=lambda x: x['x0'])
            
            # Extract the text from the row
            row_text = ' '.join(word['text'] for word in row_words)
            
            # Skip empty rows
            if not row_text.strip():
                continue
                
            # Check if this is a category row (all caps)
            if row_text.isupper() and len(row_text) > 3:
                current_category = row_text
                continue
                
            # Split the row into columns based on spacing
            # This is a simplified approach - you might need to adjust the splitting logic
            columns = re.split(r'\s{2,}', row_text)
            
            # Clean up the columns
            columns = [col.strip() for col in columns if col.strip()]
            
            if columns:
                # Add the category if we have one
                if current_category:
                    columns.insert(0, current_category)
                data.append(columns)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Clean up the DataFrame
        # Remove rows that are just headers or footers
        df = df[~df[0].str.contains('Page|Uses PA/PDL|Exemption Form', na=False)]
        
        # Forward fill category values
        df[0] = df[0].fillna(method='ffill')
        
        return df

# Extract tables
pdf_path = 'WI.pdf'
df = extract_tables_from_pdf(pdf_path)

# Save to CSV
output_path = 'page_2_tables.csv'
df.to_csv(output_path, index=False)

# Display the first few rows to verify
print("First few rows of extracted data:")
print(df.head())

First few rows of extracted data:
                            0                             1  \
0               (Gen-Flector)  tramadol ER tab (Gen-Ryzolt)   
1  benzoyl peroxide OTC 2.5%,                          None   
2                     SCN  P                        5%, 10%   
3                   NP    NP        (Gen-Pennsaid solution)   
4                   NP    NP   clindamycin/benzoyl peroxide   

                           2               3     4                  5     6  \
0                         NP  (Gen-Androgel)  None               None  None   
1                       None            None  None               None  None   
2   diclofenac 1.5% solution    Belbuca Film    NP  testosterone pump  None   
3  (Gen-Axiron and Fortesta)            None  None               None  None   
4                     Conzip             SCN    NP               None  None   

      7  
0  None  
1  None  
2  None  
3  None  
4  None  


In [23]:
import pdfplumber
import pandas as pd

def analyze_pdf_structure(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Get the page dimensions
        print(f"Page dimensions: {page.width} x {page.height}")
        
        # Extract text with exact positioning
        text = page.extract_text()
        print("\nFirst 500 characters of text:")
        print(text[:500])
        
        # Extract words with their exact positions
        words = page.extract_words(
            x_tolerance=3,
            y_tolerance=3,
            keep_blank_chars=True,
            use_text_flow=True,
            horizontal_ltr=True
        )
        
        # Print first 20 words with their positions
        print("\nFirst 20 words with positions:")
        for word in words[:20]:
            print(f"Text: {word['text']:<30} x0: {word['x0']:<8.2f} x1: {word['x1']:<8.2f} top: {word['top']:<8.2f}")
        
        # Try to identify column boundaries
        x_positions = sorted(set(word['x0'] for word in words))
        print("\nPotential column boundaries (x positions):")
        for x in x_positions[:10]:  # Print first 10 positions
            print(f"x: {x:.2f}")

# Run the analysis
pdf_path = 'WI.pdf'
analyze_pdf_structure(pdf_path)

Page dimensions: 792.0 x 612.0

First 500 characters of text:
Wisconsin Medicaid, BadgerCare Plus Standard, and SeniorCare Preferred Drug List – Quick Reference
Revised 05/29/2025 Effective 05/01/2025
Acne Agents, Topical Analgesics/Anesthetics, Topical (cont) Analgesics, Opioids Long-Acting (cont) Androgenic Agents (cont)
adapalene 0.1% cream P lidocaine 5% ointment P oxycodone ER NP Depo-testosterone* P
adapalene OTC 0.1% gel P lidocaine 5% trans patch P oxymorphone ER NP methyltestosterone capsule NP
adapalene 0.3% gel P diclofenac 1.3% patch tramadol E

First 20 words with positions:
Text: Wisconsin Medicaid, BadgerCare Plus Standard, and SeniorCare Preferred Drug List – Quick Reference  x0: 52.08    x1: 743.83   top: 22.32   
Text: Revised 05/29/2025 Effective 05/01/2025  x0: 292.32   x1: 502.78   top: 37.51   
Text:                                x0: 396.00   x1: 398.25   top: 49.80   
Text: Page 2 of 13                   x0: 702.60   x1: 768.34   top: 519.79  
Text:             

In [25]:
import pdfplumber
import pandas as pd

def extract_lines_from_pdf(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Extract text line by line
        text = page.extract_text()
        lines = text.split('\n')
        
        # Process each line
        data = []
        current_category = None
        
        for line in lines:
            # Skip header and footer lines
            if 'Page' in line or 'Uses PA/PDL' in line or 'Exemption Form' in line:
                continue
                
            # Skip empty lines
            if not line.strip():
                continue
            
            # Check if this is a category line (all caps)
            if line.isupper() and len(line) > 3:
                current_category = line
                continue
            
            # Split the line into columns (approximately)
            # The line should have 4 columns separated by spaces
            parts = line.split()
            
            # Initialize columns
            columns = [''] * 4
            
            # Try to distribute the parts into columns
            current_col = 0
            current_text = []
            
            for part in parts:
                # If we see a P or NP, it's likely the end of a column
                if part in ['P', 'NP', 'SCN']:
                    current_text.append(part)
                    columns[current_col] = ' '.join(current_text)
                    current_col += 1
                    current_text = []
                else:
                    current_text.append(part)
            
            # Add any remaining text to the last column
            if current_text:
                columns[current_col] = ' '.join(current_text)
            
            # Add the category if we have one
            if current_category:
                columns.insert(0, current_category)
            
            # Add the row to our data
            data.append(columns)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Clean up the DataFrame
        # Remove rows where all columns are empty
        df = df.dropna(how='all')
        
        # Forward fill category values
        df[0] = df[0].fillna(method='ffill')
        
        return df

# Extract lines
pdf_path = 'WI.pdf'
df = extract_lines_from_pdf(pdf_path)

# Save to CSV
output_path = 'page_2_tables.csv'
df.to_csv(output_path, index=False)

# Display the first few rows to verify
print("First few rows of extracted data:")
print(df.head())

IndexError: list assignment index out of range

In [ ]:
import pdfplumber
import pandas as pd

def extract_lines_from_pdf(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Extract text line by line
        text = page.extract_text()
        lines = text.split('\n')
        
        # Process each line
        data = []
        current_category = None
        
        for line in lines:
            # Skip header and footer lines
            if 'Page' in line or 'Uses PA/PDL' in line or 'Exemption Form' in line:
                continue
                
            # Skip empty lines
            if not line.strip():
                continue
            
            # Check if this is a category line (all caps)
            if line.isupper() and len(line) > 3:
                current_category = line
                continue
            
            # Split the line into columns (approximately)
            # The line should have 4 columns separated by spaces
            parts = line.split()
            
            # Initialize columns
            columns = [''] * 4
            
            # Try to distribute the parts into columns
            current_col = 0
            current_text = []
            
            for part in parts:
                # If we see a P or NP, it's likely the end of a column
                if part in ['P', 'NP', 'SCN']:
                    current_text.append(part)
                    columns[current_col] = ' '.join(current_text)
                    current_col += 1
                    current_text = []
                else:
                    current_text.append(part)
            
            # Add any remaining text to the last column
            if current_text:
                columns[current_col] = ' '.join(current_text)
            
            # Add the category if we have one
            if current_category:
                columns.insert(0, current_category)
            
            # Add the row to our data
            data.append(columns)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Clean up the DataFrame
        # Remove rows where all columns are empty
        df = df.dropna(how='all')
        
        # Forward fill category values
        df[0] = df[0].fillna(method='ffill')
        
        return df

# Extract lines
pdf_path = 'WI.pdf'
df = extract_lines_from_pdf(pdf_path)

# Save to CSV
output_path = 'page_2_tables.csv'
df.to_csv(output_path, index=False)

# Display the first few rows to verify
print("First few rows of extracted data:")
print(df.head())

IndexError: list assignment index out of range

In [ ]:
import pdfplumber
import pandas as pd

def extract_lines_from_pdf(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Extract text line by line
        text = page.extract_text()
        lines = text.split('\n')
        
        # Process each line
        data = []
        current_category = None
        
        for line in lines:
            # Skip header and footer lines
            if 'Page' in line or 'Uses PA/PDL' in line or 'Exemption Form' in line:
                continue
                
            # Skip empty lines
            if not line.strip():
                continue
            
            # Check if this is a category line (all caps)
            if line.isupper() and len(line) > 3:
                current_category = line
                continue
            
            # Split the line into columns (approximately)
            # The line should have 4 columns separated by spaces
            parts = line.split()
            
            # Initialize columns
            columns = [''] * 4
            
            # Try to distribute the parts into columns
            current_col = 0
            current_text = []
            
            for part in parts:
                # If we see a P or NP, it's likely the end of a column
                if part in ['P', 'NP', 'SCN']:
                    current_text.append(part)
                    columns[current_col] = ' '.join(current_text)
                    current_col += 1
                    current_text = []
                else:
                    current_text.append(part)
            
            # Add any remaining text to the last column
            if current_text:
                columns[current_col] = ' '.join(current_text)
            
            # Add the category if we have one
            if current_category:
                columns.insert(0, current_category)
            
            # Add the row to our data
            data.append(columns)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Clean up the DataFrame
        # Remove rows where all columns are empty
        df = df.dropna(how='all')
        
        # Forward fill category values
        df[0] = df[0].fillna(method='ffill')
        
        return df

# Extract lines
pdf_path = 'WI.pdf'
df = extract_lines_from_pdf(pdf_path)

# Save to CSV
output_path = 'page_2_tables.csv'
df.to_csv(output_path, index=False)

# Display the first few rows to verify
print("First few rows of extracted data:")
print(df.head())

IndexError: list assignment index out of range

In [ ]:
import pdfplumber
import pandas as pd

def extract_lines_from_pdf(pdf_path, page_number=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number-1]
        
        # Extract text line by line
        text = page.extract_text()
        lines = text.split('\n')
        
        # Process each line
        data = []
        current_category = None
        
        for line in lines:
            # Skip header and footer lines
            if 'Page' in line or 'Uses PA/PDL' in line or 'Exemption Form' in line:
                continue
                
            # Skip empty lines
            if not line.strip():
                continue
            
            # Check if this is a category line (all caps)
            if line.isupper() and len(line) > 3:
                current_category = line
                continue
            
            # Split the line into columns (approximately)
            # The line should have 4 columns separated by spaces
            parts = line.split()
            
            # Initialize columns
            columns = [''] * 4
            
            # Try to distribute the parts into columns
            current_col = 0
            current_text = []
            
            for part in parts:
                # If we see a P or NP, it's likely the end of a column
                if part in ['P', 'NP', 'SCN']:
                    current_text.append(part)
                    columns[current_col] = ' '.join(current_text)
                    current_col += 1
                    current_text = []
                else:
                    current_text.append(part)
            
            # Add any remaining text to the last column
            if current_text:
                columns[current_col] = ' '.join(current_text)
            
            # Add the category if we have one
            if current_category:
                columns.insert(0, current_category)
            
            # Add the row to our data
            data.append(columns)
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Clean up the DataFrame
        # Remove rows where all columns are empty
        df = df.dropna(how='all')
        
        # Forward fill category values
        df[0] = df[0].fillna(method='ffill')
        
        return df

# Extract lines
pdf_path = 'WI.pdf'
df = extract_lines_from_pdf(pdf_path)

# Save to CSV
output_path = 'page_2_tables.csv'
df.to_csv(output_path, index=False)

# Display the first few rows to verify
print("First few rows of extracted data:")
print(df.head())

IndexError: list assignment index out of range